In [1]:
# Generic inputs for most ML tasks
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn import tree
import graphviz
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
import xgboost as xgb

pd.options.display.float_format = '{:,.2f}'.format
pd.set_option('display.max_columns', None)

# setup interactive notebook mode
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from IPython.display import display, HTML

#### Read, pre-process and visualize data

In [2]:
# fetch data 
# flight_data = pd.read_csv('./dataset/bts_data/flight_jan_jun_22.csv', parse_dates='FL_DATE')
flight_data = pd.read_csv('./dataset/bts_data/flight_jan_jun_22.csv')
flight_data.shape

# Convert 'FL_DATE' to datetime column
flight_data['FL_DATE'] = pd.to_datetime(flight_data['FL_DATE'].str[:-4])

# Append leading zero to time columns. ex 23 -> '0023'
flight_data['CRS_DEP_TIME'] = flight_data['CRS_DEP_TIME'].astype(str).apply(lambda x: x.zfill(4))
flight_data['CRS_ARR_TIME'] = flight_data['CRS_ARR_TIME'].astype(str).apply(lambda x: x.zfill(4))

# Calculate scheduled departure and arrival times
flight_data['SCH_DEP_TIME'] = flight_data.apply(lambda row: row['FL_DATE'] + pd.Timedelta(hours=int(row['CRS_DEP_TIME'][:2]), minutes=int(row['CRS_DEP_TIME'][2:])), axis=1)
flight_data['SCH_ARR_TIME'] = flight_data.apply(lambda row: row['FL_DATE'] + pd.Timedelta(hours=int(row['CRS_ARR_TIME'][:2]), minutes=int(row['CRS_ARR_TIME'][2:])), axis=1)

# Drop unnecessary columns
flight_data.drop(columns=['FL_DATE', 'CRS_DEP_TIME', 'CRS_ARR_TIME', 'MONTH'], axis=1, inplace=True)
flight_data.head()
flight_data.shape


(1751, 21)

/var/folders/6b/tmvvscrj5wb5r4ngnndf1gtm0000gn/T/ipykernel_18128/153409611.py:7: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  flight_data['FL_DATE'] = pd.to_datetime(flight_data['FL_DATE'].str[:-4])


,DAY_OF_WEEK,MKT_UNIQUE_CARRIER,MKT_CARRIER_FL_NUM,OP_UNIQUE_CARRIER,OP_CARRIER_FL_NUM,ORIGIN,DEST,ARR_TIME,ARR_DELAY,CANCELLED,DIVERTED,DUP,DISTANCE,WEATHER_DELAY,NAS_DELAY,LATE_AIRCRAFT_DELAY,DIV_AIRPORT_LANDINGS,SCH_DEP_TIME,SCH_ARR_TIME
0,1,AA,4134,MQ,4134,ORD,SYR,"1,257.00",-19.00,0,0,N,607,NaN,NaN,NaN,0,2022-01-03 10:25:00,2022-01-03 13:16:00
1,1,AA,4253,MQ,4253,ORD,SYR,"2,024.00",12.00,0,0,N,607,NaN,NaN,NaN,0,2022-01-03 17:25:00,2022-01-03 20:12:00
2,1,AA,4316,MQ,4316,ORD,SYR,"1,813.00",-2.00,0,0,N,607,NaN,NaN,NaN,0,2022-01-03 15:28:00,2022-01-03 18:15:00
3,1,B6,116,B6,116,JFK,SYR,NaN,NaN,1,0,N,209,NaN,NaN,NaN,0,2022-01-03 08:29:00,2022-01-03 09:50:00
4,1,B6,2516,B6,2516,JFK,SYR,"2,338.00",39.00,0,0,N,209,0.00,0.00,0.00,0,2022-01-03 21:45:00,2022-01-03 22:59:00


(1751, 19)

In [3]:
# Now we filter out all flights which arrive to Syracuse Airport on next Day.
# i.e. Remove all late night flights

# Negative "diff_arrival_departure" indicates that flight arrive next day.
diff_arrival_departure = flight_data['SCH_ARR_TIME'] - flight_data['SCH_DEP_TIME']


flight_data = flight_data[diff_arrival_departure > pd.Timedelta(0)]

flight_data.shape
flight_data.dtypes

(1717, 19)

DAY_OF_WEEK                      int64
MKT_UNIQUE_CARRIER              object
MKT_CARRIER_FL_NUM               int64
OP_UNIQUE_CARRIER               object
OP_CARRIER_FL_NUM                int64
ORIGIN                          object
DEST                            object
ARR_TIME                       float64
ARR_DELAY                      float64
CANCELLED                        int64
DIVERTED                         int64
DUP                             object
DISTANCE                         int64
WEATHER_DELAY                  float64
NAS_DELAY                      float64
LATE_AIRCRAFT_DELAY            float64
DIV_AIRPORT_LANDINGS             int64
SCH_DEP_TIME            datetime64[ns]
SCH_ARR_TIME            datetime64[ns]
dtype: object

In [4]:
weather_data = pd.read_csv('./dataset/weather_data/merged_weather_hourly_2022_to_2023.csv', parse_dates=['datetime'])
weather_data.head()
weather_data.shape
weather_data.dtypes

,name,datetime,temp,feelslike,dew,humidity,precip,precipprob,preciptype,snow,snowdepth,windgust,windspeed,winddir,sealevelpressure,cloudcover,visibility,solarradiation,solarenergy,uvindex,severerisk,conditions,icon,stations
0,ORD,2022-01-01 00:00:00,40.90,33.40,36.50,84.14,0.00,0,NaN,0.00,0.00,21.90,13.80,12.00,"1,006.70",100.00,5.40,0,0.00,0,NaN,Overcast,cloudy,"72534014819,KORD,KMDW,72530094846,74466504838,..."
1,ORD,2022-01-01 01:00:00,39.50,30.80,36.00,86.96,0.00,0,NaN,0.00,0.00,26.20,16.30,15.00,"1,007.10",100.00,3.30,0,0.00,0,NaN,Overcast,cloudy,"72534014819,KORD,KMDW,72530094846,74466504838,..."
2,ORD,2022-01-01 02:00:00,38.30,31.40,34.70,86.89,0.00,0,NaN,0.00,0.00,NaN,10.40,5.00,"1,007.90",100.00,3.30,0,0.00,0,NaN,Overcast,cloudy,"72534014819,KORD,KMDW,72530094846,74466504838,..."
3,ORD,2022-01-01 03:00:00,38.50,30.80,34.40,85.10,0.00,0,NaN,0.00,0.00,NaN,12.40,11.00,"1,008.50",100.00,5.60,0,0.00,0,NaN,Overcast,cloudy,"72534014819,KORD,KMDW,72530094846,74466504838,..."
4,ORD,2022-01-01 04:00:00,38.30,29.90,33.50,82.72,0.00,0,NaN,0.00,0.00,26.40,14.10,10.00,"1,008.40",100.00,8.40,0,0.00,0,NaN,Overcast,cloudy,"72534014819,KORD,KMDW,72530094846,74466504838,..."


(70080, 24)

name                        object
datetime            datetime64[ns]
temp                       float64
feelslike                  float64
dew                        float64
humidity                   float64
precip                     float64
precipprob                   int64
preciptype                  object
snow                       float64
snowdepth                  float64
windgust                   float64
windspeed                  float64
winddir                    float64
sealevelpressure           float64
cloudcover                 float64
visibility                 float64
solarradiation               int64
solarenergy                float64
uvindex                      int64
severerisk                 float64
conditions                  object
icon                        object
stations                    object
dtype: object

## Merging Flight and Weather Data

We're combining weather and airline data, needing both origin and destination weather information.

### Steps for Merging:

- **Load Weather Data:** Load two data frames for weather data, one for origin and one for destination.
- **Column Renaming:** Rename origin columns with prefix `ORGIN_WTH_` and destination columns with prefix `DEST_WTH_`.
- **Merge Origin and Destination Weather:** Merge origin and destination weather data one by one.
- **Create Join Columns:** Add join columns in both datasets.
    - **For Origin:**
        - Flight data `ORGIN_WTH_JOIN` = `SCH_DEP_TIME` (rounded to nearest hour) + `ORIGIN`
        - Origin Weather data `ORGIN_WTH_JOIN` = `ORGIN_WTH_datetime` + `name`
    - **For Destination:**
        - Flight data `DEST_WTH_JOIN` = `SCH_ARR_TIME` (rounded to nearest hour) + `ORIGIN`
        - Destination Weather data `DEST_WTH_JOIN` = `DEST_WTH_datetime` + `name`
- **Final Merge:** Join origin and destination data sequentially. Drop the join columns added previously.


In [5]:
# Create two data frame for origin and destination. Also rename the columns by adding the prefix.
rename_origin = {}
rename_dest = {}
for col in weather_data.columns:
    rename_origin[col] = 'ORGIN_WTH_' + col
    rename_dest[col] = 'DEST_WTH_' + col
    
org_weather_data = weather_data.rename(columns=rename_origin)
dst_weather_data = weather_data.rename(columns=rename_dest)

In [6]:
# Adding Join columns
flight_data['ORGIN_WTH_JOIN'] = flight_data['SCH_DEP_TIME'].dt.round('H').astype(str) + flight_data['ORIGIN']
flight_data['DEST_WTH_JOIN'] = flight_data['SCH_ARR_TIME'].dt.round('H').astype(str) + flight_data['DEST']
org_weather_data['ORGIN_WTH_JOIN'] = org_weather_data['ORGIN_WTH_datetime'].astype(str) + org_weather_data['ORGIN_WTH_name']
dst_weather_data['DEST_WTH_JOIN'] = dst_weather_data['DEST_WTH_datetime'].astype(str) + dst_weather_data['DEST_WTH_name']
flight_data.head()
org_weather_data.head()
dst_weather_data.head()

,DAY_OF_WEEK,MKT_UNIQUE_CARRIER,MKT_CARRIER_FL_NUM,OP_UNIQUE_CARRIER,OP_CARRIER_FL_NUM,ORIGIN,DEST,ARR_TIME,ARR_DELAY,CANCELLED,DIVERTED,DUP,DISTANCE,WEATHER_DELAY,NAS_DELAY,LATE_AIRCRAFT_DELAY,DIV_AIRPORT_LANDINGS,SCH_DEP_TIME,SCH_ARR_TIME,ORGIN_WTH_JOIN,DEST_WTH_JOIN
0,1,AA,4134,MQ,4134,ORD,SYR,"1,257.00",-19.00,0,0,N,607,NaN,NaN,NaN,0,2022-01-03 10:25:00,2022-01-03 13:16:00,2022-01-03 10:00:00ORD,2022-01-03 13:00:00SYR
1,1,AA,4253,MQ,4253,ORD,SYR,"2,024.00",12.00,0,0,N,607,NaN,NaN,NaN,0,2022-01-03 17:25:00,2022-01-03 20:12:00,2022-01-03 17:00:00ORD,2022-01-03 20:00:00SYR
2,1,AA,4316,MQ,4316,ORD,SYR,"1,813.00",-2.00,0,0,N,607,NaN,NaN,NaN,0,2022-01-03 15:28:00,2022-01-03 18:15:00,2022-01-03 15:00:00ORD,2022-01-03 18:00:00SYR
3,1,B6,116,B6,116,JFK,SYR,NaN,NaN,1,0,N,209,NaN,NaN,NaN,0,2022-01-03 08:29:00,2022-01-03 09:50:00,2022-01-03 08:00:00JFK,2022-01-03 10:00:00SYR
4,1,B6,2516,B6,2516,JFK,SYR,"2,338.00",39.00,0,0,N,209,0.00,0.00,0.00,0,2022-01-03 21:45:00,2022-01-03 22:59:00,2022-01-03 22:00:00JFK,2022-01-03 23:00:00SYR


,ORGIN_WTH_name,ORGIN_WTH_datetime,ORGIN_WTH_temp,ORGIN_WTH_feelslike,ORGIN_WTH_dew,ORGIN_WTH_humidity,ORGIN_WTH_precip,ORGIN_WTH_precipprob,ORGIN_WTH_preciptype,ORGIN_WTH_snow,ORGIN_WTH_snowdepth,ORGIN_WTH_windgust,ORGIN_WTH_windspeed,ORGIN_WTH_winddir,ORGIN_WTH_sealevelpressure,ORGIN_WTH_cloudcover,ORGIN_WTH_visibility,ORGIN_WTH_solarradiation,ORGIN_WTH_solarenergy,ORGIN_WTH_uvindex,ORGIN_WTH_severerisk,ORGIN_WTH_conditions,ORGIN_WTH_icon,ORGIN_WTH_stations,ORGIN_WTH_JOIN
0,ORD,2022-01-01 00:00:00,40.90,33.40,36.50,84.14,0.00,0,NaN,0.00,0.00,21.90,13.80,12.00,"1,006.70",100.00,5.40,0,0.00,0,NaN,Overcast,cloudy,"72534014819,KORD,KMDW,72530094846,74466504838,...",2022-01-01 00:00:00ORD
1,ORD,2022-01-01 01:00:00,39.50,30.80,36.00,86.96,0.00,0,NaN,0.00,0.00,26.20,16.30,15.00,"1,007.10",100.00,3.30,0,0.00,0,NaN,Overcast,cloudy,"72534014819,KORD,KMDW,72530094846,74466504838,...",2022-01-01 01:00:00ORD
2,ORD,2022-01-01 02:00:00,38.30,31.40,34.70,86.89,0.00,0,NaN,0.00,0.00,NaN,10.40,5.00,"1,007.90",100.00,3.30,0,0.00,0,NaN,Overcast,cloudy,"72534014819,KORD,KMDW,72530094846,74466504838,...",2022-01-01 02:00:00ORD
3,ORD,2022-01-01 03:00:00,38.50,30.80,34.40,85.10,0.00,0,NaN,0.00,0.00,NaN,12.40,11.00,"1,008.50",100.00,5.60,0,0.00,0,NaN,Overcast,cloudy,"72534014819,KORD,KMDW,72530094846,74466504838,...",2022-01-01 03:00:00ORD
4,ORD,2022-01-01 04:00:00,38.30,29.90,33.50,82.72,0.00,0,NaN,0.00,0.00,26.40,14.10,10.00,"1,008.40",100.00,8.40,0,0.00,0,NaN,Overcast,cloudy,"72534014819,KORD,KMDW,72530094846,74466504838,...",2022-01-01 04:00:00ORD


,DEST_WTH_name,DEST_WTH_datetime,DEST_WTH_temp,DEST_WTH_feelslike,DEST_WTH_dew,DEST_WTH_humidity,DEST_WTH_precip,DEST_WTH_precipprob,DEST_WTH_preciptype,DEST_WTH_snow,DEST_WTH_snowdepth,DEST_WTH_windgust,DEST_WTH_windspeed,DEST_WTH_winddir,DEST_WTH_sealevelpressure,DEST_WTH_cloudcover,DEST_WTH_visibility,DEST_WTH_solarradiation,DEST_WTH_solarenergy,DEST_WTH_uvindex,DEST_WTH_severerisk,DEST_WTH_conditions,DEST_WTH_icon,DEST_WTH_stations,DEST_WTH_JOIN
0,ORD,2022-01-01 00:00:00,40.90,33.40,36.50,84.14,0.00,0,NaN,0.00,0.00,21.90,13.80,12.00,"1,006.70",100.00,5.40,0,0.00,0,NaN,Overcast,cloudy,"72534014819,KORD,KMDW,72530094846,74466504838,...",2022-01-01 00:00:00ORD
1,ORD,2022-01-01 01:00:00,39.50,30.80,36.00,86.96,0.00,0,NaN,0.00,0.00,26.20,16.30,15.00,"1,007.10",100.00,3.30,0,0.00,0,NaN,Overcast,cloudy,"72534014819,KORD,KMDW,72530094846,74466504838,...",2022-01-01 01:00:00ORD
2,ORD,2022-01-01 02:00:00,38.30,31.40,34.70,86.89,0.00,0,NaN,0.00,0.00,NaN,10.40,5.00,"1,007.90",100.00,3.30,0,0.00,0,NaN,Overcast,cloudy,"72534014819,KORD,KMDW,72530094846,74466504838,...",2022-01-01 02:00:00ORD
3,ORD,2022-01-01 03:00:00,38.50,30.80,34.40,85.10,0.00,0,NaN,0.00,0.00,NaN,12.40,11.00,"1,008.50",100.00,5.60,0,0.00,0,NaN,Overcast,cloudy,"72534014819,KORD,KMDW,72530094846,74466504838,...",2022-01-01 03:00:00ORD
4,ORD,2022-01-01 04:00:00,38.30,29.90,33.50,82.72,0.00,0,NaN,0.00,0.00,26.40,14.10,10.00,"1,008.40",100.00,8.40,0,0.00,0,NaN,Overcast,cloudy,"72534014819,KORD,KMDW,72530094846,74466504838,...",2022-01-01 04:00:00ORD


In [7]:
# Join data set
merged_flight_weather = pd.merge(flight_data, org_weather_data, on='ORGIN_WTH_JOIN')
merged_flight_weather = pd.merge(merged_flight_weather, dst_weather_data, on='DEST_WTH_JOIN')

merged_flight_weather.head()
merged_flight_weather.shape
merged_flight_weather.columns

,DAY_OF_WEEK,MKT_UNIQUE_CARRIER,MKT_CARRIER_FL_NUM,OP_UNIQUE_CARRIER,OP_CARRIER_FL_NUM,ORIGIN,DEST,ARR_TIME,ARR_DELAY,CANCELLED,DIVERTED,DUP,DISTANCE,WEATHER_DELAY,NAS_DELAY,LATE_AIRCRAFT_DELAY,DIV_AIRPORT_LANDINGS,SCH_DEP_TIME,SCH_ARR_TIME,ORGIN_WTH_JOIN,DEST_WTH_JOIN,ORGIN_WTH_name,ORGIN_WTH_datetime,ORGIN_WTH_temp,ORGIN_WTH_feelslike,ORGIN_WTH_dew,ORGIN_WTH_humidity,ORGIN_WTH_precip,ORGIN_WTH_precipprob,ORGIN_WTH_preciptype,ORGIN_WTH_snow,ORGIN_WTH_snowdepth,ORGIN_WTH_windgust,ORGIN_WTH_windspeed,ORGIN_WTH_winddir,ORGIN_WTH_sealevelpressure,ORGIN_WTH_cloudcover,ORGIN_WTH_visibility,ORGIN_WTH_solarradiation,ORGIN_WTH_solarenergy,ORGIN_WTH_uvindex,ORGIN_WTH_severerisk,ORGIN_WTH_conditions,ORGIN_WTH_icon,ORGIN_WTH_stations,DEST_WTH_name,DEST_WTH_datetime,DEST_WTH_temp,DEST_WTH_feelslike,DEST_WTH_dew,DEST_WTH_humidity,DEST_WTH_precip,DEST_WTH_precipprob,DEST_WTH_preciptype,DEST_WTH_snow,DEST_WTH_snowdepth,DEST_WTH_windgust,DEST_WTH_windspeed,DEST_WTH_winddir,DEST_WTH_sealevelpressure,DEST_WTH_cloudcover,DEST_WTH_visibility,DEST_WTH_solarradiation,DEST_WTH_solarenergy,DEST_WTH_uvindex,DEST_WTH_severerisk,DEST_WTH_conditions,DEST_WTH_icon,DEST_WTH_stations
0,1,AA,4134,MQ,4134,ORD,SYR,"1,257.00",-19.00,0,0,N,607,NaN,NaN,NaN,0,2022-01-03 10:25:00,2022-01-03 13:16:00,2022-01-03 10:00:00ORD,2022-01-03 13:00:00SYR,ORD,2022-01-03 10:00:00,14.10,3.30,3.30,61.41,0.00,0,NaN,0.00,2.50,NaN,7.80,226.00,"1,030.30",0.00,9.90,135,0.50,1,NaN,Clear,clear-day,"72534014819,KORD,KMDW,72530094846,F1983,744665...",SYR,2022-01-03 13:00:00,17.80,8.50,6.20,59.66,0.00,0,NaN,0.00,0.13,NaN,6.90,309.00,"1,025.20",84.30,9.90,297,1.10,3,NaN,Partially cloudy,partly-cloudy-day,"KSYR,72519014771,72514654773,AS511,KFZY,725196..."
1,1,AA,4253,MQ,4253,ORD,SYR,"2,024.00",12.00,0,0,N,607,NaN,NaN,NaN,0,2022-01-03 17:25:00,2022-01-03 20:12:00,2022-01-03 17:00:00ORD,2022-01-03 20:00:00SYR,ORD,2022-01-03 17:00:00,20.10,9.30,5.00,51.38,0.00,0,NaN,0.00,2.45,NaN,9.60,205.00,"1,028.20",24.20,9.90,9,0.00,0,NaN,Partially cloudy,partly-cloudy-night,"72534014819,KORD,KMDW,72530094846,F1983,744665...",SYR,2022-01-03 20:00:00,12.70,3.50,6.90,77.32,0.00,0,NaN,0.00,0.10,NaN,5.80,335.00,"1,027.50",29.60,9.90,0,0.00,0,NaN,Partially cloudy,partly-cloudy-night,"KSYR,72519014771,72514654773,KFZY,72519664775,..."
2,1,AA,4316,MQ,4316,ORD,SYR,"1,813.00",-2.00,0,0,N,607,NaN,NaN,NaN,0,2022-01-03 15:28:00,2022-01-03 18:15:00,2022-01-03 15:00:00ORD,2022-01-03 18:00:00SYR,ORD,2022-01-03 15:00:00,22.70,10.90,3.30,42.51,0.00,0,NaN,0.00,2.46,NaN,12.50,218.00,"1,028.10",24.20,9.90,218,0.80,2,NaN,Partially cloudy,partly-cloudy-day,"72534014819,KORD,KMDW,72530094846,F1983,744665...",SYR,2022-01-03 18:00:00,15.00,6.30,8.70,75.56,0.00,0,NaN,0.00,0.11,NaN,5.70,292.00,"1,026.40",47.30,9.90,0,0.00,0,NaN,Partially cloudy,partly-cloudy-night,"KSYR,72519014771,72514654773,KFZY,72519664775,..."
3,1,UA,4576,G7,4576,ORD,SYR,"1,722.00",-14.00,0,0,N,607,NaN,NaN,NaN,0,2022-01-03 14:45:00,2022-01-03 17:36:00,2022-01-03 15:00:00ORD,2022-01-03 18:00:00SYR,ORD,2022-01-03 15:00:00,22.70,10.90,3.30,42.51,0.00,0,NaN,0.00,2.46,NaN,12.50,218.00,"1,028.10",24.20,9.90,218,0.80,2,NaN,Partially cloudy,partly-cloudy-day,"72534014819,KORD,KMDW,72530094846,F1983,744665...",SYR,2022-01-03 18:00:00,15.00,6.30,8.70,75.56,0.00,0,NaN,0.00,0.11,NaN,5.70,292.00,"1,026.40",47.30,9.90,0,0.00,0,NaN,Partially cloudy,partly-cloudy-night,"KSYR,72519014771,72514654773,KFZY,72519664775,..."
4,1,B6,116,B6,116,JFK,SYR,NaN,NaN,1,0,N,209,NaN,NaN,NaN,0,2022-01-03 08:29:00,2022-01-03 09:50:00,2022-01-03 08:00:00JFK,2022-01-03 10:00:00SYR,JFK,2022-01-03 08:00:00,31.10,22.10,13.50,47.57,0.00,0,NaN,0.00,0.00,20.20,11.20,10.00,"1,019.60",46.60,9.90,8,0.00,0,NaN,Partially cloudy,partly-cloudy-day,"72505394728,KLGA,F1417,KNYC,72503014732",SYR,2022-01-03 10:00:00,14.90,5.10,8.70,75.92,0.00,0,NaN,0.00,0.13,NaN,6.80,301.00,"1,027.10",84.30,9.90,103,0.40,1,NaN,Partially cloudy,partly-cloudy-day,"KSYR,72519014771,72514654773,AS511,KFZY,725196..."


(1717, 69)

Index(['DAY_OF_WEEK', 'MKT_UNIQUE_CARRIER', 'MKT_CARRIER_FL_NUM',
       'OP_UNIQUE_CARRIER', 'OP_CARRIER_FL_NUM', 'ORIGIN', 'DEST', 'ARR_TIME',
       'ARR_DELAY', 'CANCELLED', 'DIVERTED', 'DUP', 'DISTANCE',
       'WEATHER_DELAY', 'NAS_DELAY', 'LATE_AIRCRAFT_DELAY',
       'DIV_AIRPORT_LANDINGS', 'SCH_DEP_TIME', 'SCH_ARR_TIME',
       'ORGIN_WTH_JOIN', 'DEST_WTH_JOIN', 'ORGIN_WTH_name',
       'ORGIN_WTH_datetime', 'ORGIN_WTH_temp', 'ORGIN_WTH_feelslike',
       'ORGIN_WTH_dew', 'ORGIN_WTH_humidity', 'ORGIN_WTH_precip',
       'ORGIN_WTH_precipprob', 'ORGIN_WTH_preciptype', 'ORGIN_WTH_snow',
       'ORGIN_WTH_snowdepth', 'ORGIN_WTH_windgust', 'ORGIN_WTH_windspeed',
       'ORGIN_WTH_winddir', 'ORGIN_WTH_sealevelpressure',
       'ORGIN_WTH_cloudcover', 'ORGIN_WTH_visibility',
       'ORGIN_WTH_solarradiation', 'ORGIN_WTH_solarenergy',
       'ORGIN_WTH_uvindex', 'ORGIN_WTH_severerisk', 'ORGIN_WTH_conditions',
       'ORGIN_WTH_icon', 'ORGIN_WTH_stations', 'DEST_WTH_name',
       'DES

In [8]:
# Drop unnecessary columns
merged_flight_weather.drop(columns=['ORGIN_WTH_JOIN', 'DEST_WTH_JOIN', 'ORGIN_WTH_datetime', 'DEST_WTH_datetime', 'ORGIN_WTH_name', 'DEST_WTH_name'])

,DAY_OF_WEEK,MKT_UNIQUE_CARRIER,MKT_CARRIER_FL_NUM,OP_UNIQUE_CARRIER,OP_CARRIER_FL_NUM,ORIGIN,DEST,ARR_TIME,ARR_DELAY,CANCELLED,DIVERTED,DUP,DISTANCE,WEATHER_DELAY,NAS_DELAY,LATE_AIRCRAFT_DELAY,DIV_AIRPORT_LANDINGS,SCH_DEP_TIME,SCH_ARR_TIME,ORGIN_WTH_temp,ORGIN_WTH_feelslike,ORGIN_WTH_dew,ORGIN_WTH_humidity,ORGIN_WTH_precip,ORGIN_WTH_precipprob,ORGIN_WTH_preciptype,ORGIN_WTH_snow,ORGIN_WTH_snowdepth,ORGIN_WTH_windgust,ORGIN_WTH_windspeed,ORGIN_WTH_winddir,ORGIN_WTH_sealevelpressure,ORGIN_WTH_cloudcover,ORGIN_WTH_visibility,ORGIN_WTH_solarradiation,ORGIN_WTH_solarenergy,ORGIN_WTH_uvindex,ORGIN_WTH_severerisk,ORGIN_WTH_conditions,ORGIN_WTH_icon,ORGIN_WTH_stations,DEST_WTH_temp,DEST_WTH_feelslike,DEST_WTH_dew,DEST_WTH_humidity,DEST_WTH_precip,DEST_WTH_precipprob,DEST_WTH_preciptype,DEST_WTH_snow,DEST_WTH_snowdepth,DEST_WTH_windgust,DEST_WTH_windspeed,DEST_WTH_winddir,DEST_WTH_sealevelpressure,DEST_WTH_cloudcover,DEST_WTH_visibility,DEST_WTH_solarradiation,DEST_WTH_solarenergy,DEST_WTH_uvindex,DEST_WTH_severerisk,DEST_WTH_conditions,DEST_WTH_icon,DEST_WTH_stations
0,1,AA,4134,MQ,4134,ORD,SYR,"1,257.00",-19.00,0,0,N,607,NaN,NaN,NaN,0,2022-01-03 10:25:00,2022-01-03 13:16:00,14.10,3.30,3.30,61.41,0.00,0,NaN,0.00,2.50,NaN,7.80,226.00,"1,030.30",0.00,9.90,135,0.50,1,NaN,Clear,clear-day,"72534014819,KORD,KMDW,72530094846,F1983,744665...",17.80,8.50,6.20,59.66,0.00,0,NaN,0.00,0.13,NaN,6.90,309.00,"1,025.20",84.30,9.90,297,1.10,3,NaN,Partially cloudy,partly-cloudy-day,"KSYR,72519014771,72514654773,AS511,KFZY,725196..."
1,1,AA,4253,MQ,4253,ORD,SYR,"2,024.00",12.00,0,0,N,607,NaN,NaN,NaN,0,2022-01-03 17:25:00,2022-01-03 20:12:00,20.10,9.30,5.00,51.38,0.00,0,NaN,0.00,2.45,NaN,9.60,205.00,"1,028.20",24.20,9.90,9,0.00,0,NaN,Partially cloudy,partly-cloudy-night,"72534014819,KORD,KMDW,72530094846,F1983,744665...",12.70,3.50,6.90,77.32,0.00,0,NaN,0.00,0.10,NaN,5.80,335.00,"1,027.50",29.60,9.90,0,0.00,0,NaN,Partially cloudy,partly-cloudy-night,"KSYR,72519014771,72514654773,KFZY,72519664775,..."
2,1,AA,4316,MQ,4316,ORD,SYR,"1,813.00",-2.00,0,0,N,607,NaN,NaN,NaN,0,2022-01-03 15:28:00,2022-01-03 18:15:00,22.70,10.90,3.30,42.51,0.00,0,NaN,0.00,2.46,NaN,12.50,218.00,"1,028.10",24.20,9.90,218,0.80,2,NaN,Partially cloudy,partly-cloudy-day,"72534014819,KORD,KMDW,72530094846,F1983,744665...",15.00,6.30,8.70,75.56,0.00,0,NaN,0.00,0.11,NaN,5.70,292.00,"1,026.40",47.30,9.90,0,0.00,0,NaN,Partially cloudy,partly-cloudy-night,"KSYR,72519014771,72514654773,KFZY,72519664775,..."
3,1,UA,4576,G7,4576,ORD,SYR,"1,722.00",-14.00,0,0,N,607,NaN,NaN,NaN,0,2022-01-03 14:45:00,2022-01-03 17:36:00,22.70,10.90,3.30,42.51,0.00,0,NaN,0.00,2.46,NaN,12.50,218.00,"1,028.10",24.20,9.90,218,0.80,2,NaN,Partially cloudy,partly-cloudy-day,"72534014819,KORD,KMDW,72530094846,F1983,744665...",15.00,6.30,8.70,75.56,0.00,0,NaN,0.00,0.11,NaN,5.70,292.00,"1,026.40",47.30,9.90,0,0.00,0,NaN,Partially cloudy,partly-cloudy-night,"KSYR,72519014771,72514654773,KFZY,72519664775,..."
4,1,B6,116,B6,116,JFK,SYR,NaN,NaN,1,0,N,209,NaN,NaN,NaN,0,2022-01-03 08:29:00,2022-01-03 09:50:00,31.10,22.10,13.50,47.57,0.00,0,NaN,0.00,0.00,20.20,11.20,10.00,"1,019.60",46.60,9.90,8,0.00,0,NaN,Partially cloudy,partly-cloudy-day,"72505394728,KLGA,F1417,KNYC,72503014732",14.90,5.10,8.70,75.92,0.00,0,NaN,0.00,0.13,NaN,6.80,301.00,"1,027.10",84.30,9.90,103,0.40,1,NaN,Partially cloudy,partly-cloudy-day,"KSYR,72519014771,72514654773,AS511,KFZY,725196..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1712,7,AA,4798,YX,4798,ORD,SYR,"2,315.00",-2.00,0,0,N,607,NaN,NaN,NaN,0,2022-06-26 20:24:00,2022-06-26 23:17:00,76.30,76.30,48.50,37.57,0.00,0,NaN,0.00,0.00,23.80,14.80,305.00,"1,017.40",78.40,9.90,19,0.10,0,10.00,Partially cloudy,partly-cloudy-day,"72534014819,KORD,KMDW,72530094846,F1983,744665...",74.00,74.00,69.20,85.10,

In [9]:
merged_flight_weather.to_csv('./dataset/merged_flight_weather_hourly_2022_2023.csv', index=False)